In [1]:
import numpy as np
# import data as data
from gensim.models import Word2Vec
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("autotagger.tsv",sep='\t')
t = df[df['description'].isna() == False]

In [3]:
t.isna().sum()

product_id        0
product_name      0
kw_name         950
description       0
dtype: int64

In [4]:
t

,product_id,product_name,kw_name,description
0,126879,Addis Flip Top 46L Black Bin,NaN,Dispose of unwanted rubbish without having to ...
6,140919,BRM PERI-PERI SPATCHCOCK CHICKEN 1.2KG,NaN,Fully cooked;\nReady in minutes;\nCooked until...
7,122149,BRM Pork Belly Ribs Bbq 2kg,ToBeRemoved,Pre-cooked marinated in bbq sauce
11,140932,Crushed Ginger 130g,NaN,Perfect for stews & curries!
12,141080,D'Usse VSOP Cognac 750ml,NaN,D'Usse VSOP Cognac 750ml
...,...,...,...,...
13570,141715,Ybarra Extra Virgin Olive Oil 1L,NaN,Olive Oil
13597,64353,Yum Yum Smooth Peanut Butter 400g,ToBeRemoved,Give your sandwich some flavour with this amaz...
13598,64353,Yum Yum Smooth Peanut Butter 400g,spread,Give your sandwich some flavour with this amaz...
13599,64353,Yum Yum Smooth Peanut Butter 400g,peanut,Give your sandwich some flavour with this amaz...


In [5]:
t = t[t['kw_name']!='ToBeRemoved']

In [6]:
data = t.copy()

In [7]:
data.shape

(1525, 4)

In [8]:
data.reset_index(drop=True,inplace=True)
data['product_name'] = data['product_name'].apply(lambda x:x.strip())

In [9]:
data['desc_and_productname'] = data['product_name'] + ' ' + data['description']

# Model

In [10]:
from gensim.parsing.preprocessing import preprocess_string

In [11]:
preprocess_string('Dispose 111/. / of unwanted played rubbish without having')

['dispos', 'unwant', 'plai', 'rubbish', 'have']

In [12]:
#https://radimrehurek.com/gensim/parsing/preprocessing.html
all_description = []
for description in data['desc_and_productname'].values:
    all_description.append(preprocess_string(description))

In [13]:
all_description

[['addi',
  'flip',
  'black',
  'bin',
  'dispos',
  'unwant',
  'rubbish',
  'have',
  'touch',
  'bin',
  'qualiti',
  'pedal',
  'bin',
  'simpli',
  'open',
  'foot',
  'drop',
  'garbag',
  'trash',
  'bag'],
 ['brm',
  'peri',
  'peri',
  'spatchcock',
  'chicken',
  'fulli',
  'cook',
  'nreadi',
  'minut',
  'ncook',
  'tender',
  'marin',
  'famou',
  'peri',
  'peri',
  'sauc',
  'nkeep',
  'refriger',
  'nbrm',
  'spatchcock',
  'chicken',
  'fulli',
  'cook',
  'peri',
  'peri',
  'sauc'],
 ['crush', 'ginger', 'perfect', 'stew', 'curri'],
 ['uss', 'vsop', 'cognac', 'uss', 'vsop', 'cognac'],
 ['dettol',
  'liquid',
  'bathroom',
  'cleaner',
  'ocean',
  'fresh',
  'ndettol',
  'liquid',
  'bathroom',
  'cleaner',
  'ocean',
  'fresh'],
 ['frozen', 'mix', 'berri', 'special', 'selecet'],
 ['glad',
  'small',
  'zipper',
  'freezer',
  'bag',
  'glad',
  'freezer',
  'bag',
  'strong',
  'suitabl',
  'freez',
  'type',
  'food',
  'includ',
  'bag',
  'tie',
  'seal',
  'fres

In [14]:
data.head(20)

,product_id,product_name,kw_name,description,desc_and_productname
0,126879,Addis Flip Top 46L Black Bin,NaN,Dispose of unwanted rubbish without having to ...,Addis Flip Top 46L Black Bin Dispose of unwant...
1,140919,BRM PERI-PERI SPATCHCOCK CHICKEN 1.2KG,NaN,Fully cooked;\nReady in minutes;\nCooked until...,BRM PERI-PERI SPATCHCOCK CHICKEN 1.2KG Fully c...
2,140932,Crushed Ginger 130g,NaN,Perfect for stews & curries!,Crushed Ginger 130g Perfect for stews & curries!
3,141080,D'Usse VSOP Cognac 750ml,NaN,D'Usse VSOP Cognac 750ml,D'Usse VSOP Cognac 750ml D'Usse VSOP Cognac 7...
4,141432,Dettol Liquid Bathroom Cleaner Ocean Fresh 500ml,NaN,\nDettol Liquid Bathroom Cleaner Ocean Fresh 5...,Dettol Liquid Bathroom Cleaner Ocean Fresh 500...
5,141000,Frozen Mixed Berries 1Kg,NaN,specially and seleceted\n,Frozen Mixed Berries 1Kg specially and selecet...
6,140252,Glad Small Zipper Freezer Bags 15,NaN,"Glad Freezer Bags are thick, strong and suitab...",Glad Small Zipper Freezer Bags 15 Glad Freezer...
7,141487,Johnnie Walker Black Label 750ml,NaN,Whisky,Johnnie Walker Black Label 750ml Whisky
8,140985,Oranges 4pk,NaN,specially and selected,Oranges 4pk specially and selected
9,140976,Qualifresh Crushed Garlic 125g,NaN,Specially and selected,Qualifresh Crushed Garlic 125g Specially and s...


In [15]:
data['desc_processed'] = all_description

In [16]:
data.head(20)

,product_id,product_name,kw_name,description,desc_and_productname,desc_processed
0,126879,Addis Flip Top 46L Black Bin,NaN,Dispose of unwanted rubbish without having to ...,Addis Flip Top 46L Black Bin Dispose of unwant...,"[addi, flip, black, bin, dispos, unwant, rubbi..."
1,140919,BRM PERI-PERI SPATCHCOCK CHICKEN 1.2KG,NaN,Fully cooked;\nReady in minutes;\nCooked until...,BRM PERI-PERI SPATCHCOCK CHICKEN 1.2KG Fully c...,"[brm, peri, peri, spatchcock, chicken, fulli, ..."
2,140932,Crushed Ginger 130g,NaN,Perfect for stews & curries!,Crushed Ginger 130g Perfect for stews & curries!,"[crush, ginger, perfect, stew, curri]"
3,141080,D'Usse VSOP Cognac 750ml,NaN,D'Usse VSOP Cognac 750ml,D'Usse VSOP Cognac 750ml D'Usse VSOP Cognac 7...,"[uss, vsop, cognac, uss, vsop, cognac]"
4,141432,Dettol Liquid Bathroom Cleaner Ocean Fresh 500ml,NaN,\nDettol Liquid Bathroom Cleaner Ocean Fresh 5...,Dettol Liquid Bathroom Cleaner Ocean Fresh 500...,"[dettol, liquid, bathroom, cleaner, ocean, fre..."
5,141000,Frozen Mixed Berries 1Kg,NaN,specially and seleceted\n,Frozen Mixed Berries 1Kg specially and selecet...,"[frozen, mix, berri, special, selecet]"
6,140252,Glad Small Zipper Freezer Bags 15,NaN,"Glad Freezer Bags are thick, strong and suitab...",Glad Small Zipper Freezer Bags 15 Glad Freezer...,"[glad, small, zipper, freezer, bag, glad, free..."
7,141487,Johnnie Walker Black Label 750ml,NaN,Whisky,Johnnie Walker Black Label 750ml Whisky,"[johnni, walker, black, label, whiski]"
8,140985,Oranges 4pk,NaN,specially and selected,Oranges 4pk specially and selected,"[orang, special, select]"
9,140976,Qualifresh Crushed Garlic 125g,NaN,Specially and selected,Qualifresh Crushed Garlic 125g Specially and s...,"[qualifresh, crush, garlic, special, select]"


In [17]:
# data['desc_processed'] = all_description
data = data[data['desc_processed'].apply(lambda x:len(x)>=1)]

In [18]:
data['kw_name_bert'] = data['kw_name']

In [19]:
data.head(20)

,product_id,product_name,kw_name,description,desc_and_productname,desc_processed,kw_name_bert
0,126879,Addis Flip Top 46L Black Bin,NaN,Dispose of unwanted rubbish without having to ...,Addis Flip Top 46L Black Bin Dispose of unwant...,"[addi, flip, black, bin, dispos, unwant, rubbi...",NaN
1,140919,BRM PERI-PERI SPATCHCOCK CHICKEN 1.2KG,NaN,Fully cooked;\nReady in minutes;\nCooked until...,BRM PERI-PERI SPATCHCOCK CHICKEN 1.2KG Fully c...,"[brm, peri, peri, spatchcock, chicken, fulli, ...",NaN
2,140932,Crushed Ginger 130g,NaN,Perfect for stews & curries!,Crushed Ginger 130g Perfect for stews & curries!,"[crush, ginger, perfect, stew, curri]",NaN
3,141080,D'Usse VSOP Cognac 750ml,NaN,D'Usse VSOP Cognac 750ml,D'Usse VSOP Cognac 750ml D'Usse VSOP Cognac 7...,"[uss, vsop, cognac, uss, vsop, cognac]",NaN
4,141432,Dettol Liquid Bathroom Cleaner Ocean Fresh 500ml,NaN,\nDettol Liquid Bathroom Cleaner Ocean Fresh 5...,Dettol Liquid Bathroom Cleaner Ocean Fresh 500...,"[dettol, liquid, bathroom, cleaner, ocean, fre...",NaN
5,141000,Frozen Mixed Berries 1Kg,NaN,specially and seleceted\n,Frozen Mixed Berries 1Kg specially and selecet...,"[frozen, mix, berri, special, selecet]",NaN
6,140252,Glad Small Zipper Freezer Bags 15,NaN,"Glad Freezer Bags are thick, strong and suitab...",Glad Small Zipper Freezer Bags 15 Glad Freezer...,"[glad, small, zipper, freezer, bag, glad, free...",NaN
7,141487,Johnnie Walker Black Label 750ml,NaN,Whisky,Johnnie Walker Black Label 750ml Whisky,"[johnni, walker, black, label, whiski]",NaN
8,140985,Oranges 4pk,NaN,specially and selected,Oranges 4pk specially and selected,"[orang, special, select]",NaN
9,140976,Qualifresh Crushed Garlic 125g,NaN,Specially and selected,Qualifresh Crushed Garlic 125g Specially and s...,"[qualifresh, crush, garlic, special, select]",NaN


In [20]:
all_description = data['desc_processed']

In [21]:
model = Word2Vec(all_description, 
                 min_count=1,   # word frequency
                 size=10,      # dimention of word embeddings
                 workers=8,     # Number of processors
                 sg=0, # cbow
                 window=5,      # Context window
                )      

In [2]:
# save
# model.wv.save_word2vec_format('w2vec_10d.txt')

# How to load:
from gensim.models import KeyedVectors
w2v = KeyedVectors.load_word2vec_format('w2vec_10d.txt')

In [3]:
w2v['white']

array([ 0.06839806, -0.89426064, -0.37528184, -0.8991656 ,  0.01007856,
        0.04904905, -1.5981627 ,  0.48229292,  0.2081241 ,  2.498666  ],
      dtype=float32)

In [8]:
'fruit' in w2v.vocab

True

In [10]:
w2v.wv.vocab

C:\Users\avitr\AppData\Local\Temp\ipykernel_14956\329026737.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  w2v.wv.vocab


{'flavour': <gensim.models.keyedvectors.Vocab at 0x23d527919a0>,
 'fresh': <gensim.models.keyedvectors.Vocab at 0x23d52791520>,
 'fruit': <gensim.models.keyedvectors.Vocab at 0x23d52291dc0>,
 'skin': <gensim.models.keyedvectors.Vocab at 0x23d52284c10>,
 'free': <gensim.models.keyedvectors.Vocab at 0x23d527960a0>,
 'tast': <gensim.models.keyedvectors.Vocab at 0x23d64f8f6d0>,
 'soft': <gensim.models.keyedvectors.Vocab at 0x23d64f8f760>,
 'babi': <gensim.models.keyedvectors.Vocab at 0x23d64f8f700>,
 'protect': <gensim.models.keyedvectors.Vocab at 0x23d6512f730>,
 'chicken': <gensim.models.keyedvectors.Vocab at 0x23d651c24c0>,
 'rang': <gensim.models.keyedvectors.Vocab at 0x23d651c2520>,
 'water': <gensim.models.keyedvectors.Vocab at 0x23d651c2580>,
 'white': <gensim.models.keyedvectors.Vocab at 0x23d651c25e0>,
 'beef': <gensim.models.keyedvectors.Vocab at 0x23d651c2640>,
 'sweet': <gensim.models.keyedvectors.Vocab at 0x23d651c26a0>,
 'cream': <gensim.models.keyedvectors.Vocab at 0x23d651c

In [25]:
# take average of all words to make it vector for description i.e. vector for tag
vector = []
for line in all_description:
    try:
        x = np.zeros(10)
        for word in line:
            x = x + w2v[word]
        vector.append(x/len(line))
    except:
        print(line)

In [26]:
data['vector'] = vector

In [27]:
data.shape

(1525, 8)

In [28]:
data

,product_id,product_name,kw_name,description,desc_and_productname,desc_processed,kw_name_bert,vector
0,126879,Addis Flip Top 46L Black Bin,NaN,Dispose of unwanted rubbish without having to ...,Addis Flip Top 46L Black Bin Dispose of unwant...,"[addi, flip, black, bin, dispos, unwant, rubbi...",NaN,"[0.17322857377002948, -0.21354466241318731, 0...."
1,140919,BRM PERI-PERI SPATCHCOCK CHICKEN 1.2KG,NaN,Fully cooked;\nReady in minutes;\nCooked until...,BRM PERI-PERI SPATCHCOCK CHICKEN 1.2KG Fully c...,"[brm, peri, peri, spatchcock, chicken, fulli, ...",NaN,"[0.24177374232273835, -0.26565936494332093, 0...."
2,140932,Crushed Ginger 130g,NaN,Perfect for stews & curries!,Crushed Ginger 130g Perfect for stews & curries!,"[crush, ginger, perfect, stew, curri]",NaN,"[0.197862619580701, -0.2505973964929581, 0.075..."
3,141080,D'Usse VSOP Cognac 750ml,NaN,D'Usse VSOP Cognac 750ml,D'Usse VSOP Cognac 750ml D'Usse VSOP Cognac 7...,"[uss, vsop, cognac, uss, vsop, cognac]",NaN,"[0.08413595954577129, -0.08483439063032468, -0..."
4,141432,Dettol Liquid Bathroom Cleaner Ocean Fresh 500ml,NaN,\nDettol Liquid Bathroom Cleaner Ocean Fresh 5...,Dettol Liquid Bathroom Cleaner Ocean Fresh 500...,"[dettol, liquid, bathroom, cleaner, ocean, fre...",NaN,"[0.26170364130909246, -0.32234730443451554, 0...."
...,...,...,...,...,...,...,...,...
1520,141502,WINGSON white spirit vinegar 375ml,NaN,WINGSON white spirit vinegar 375ml,WINGSON white spirit vinegar 375ml WINGSON whi...,"[wingson, white, spirit, vinegar, wingson, whi...",NaN,"[0.27126368321478367, -0.31131810415536165, 0...."
1521,141715,Ybarra Extra Virgin Olive Oil 1L,NaN,Olive Oil,Ybarra Extra Virgin Olive Oil 1L Olive Oil,"[ybarra, extra, virgin, oliv, oil, oliv, oil]",NaN,"[0.2824192265314715, -0.36693695054522585, 0.0..."
1522,64353,Yum Yum Smooth Peanut Butter 400g,spread,Give your sandwich some flavour with this amaz...,Yum Yum Smooth Peanut Butter 400g Give your sa...,"[yum, yum, smooth, peanut, butter, sandwich, f...",spread,"[0.26763691306114196, -0.3053634285926819, 0.0..."
1523,64353,Yum Yum Smooth Peanut Butter 400g,peanut,Give your sandwich some flavour with this amaz...,Yum Yum Smooth Peanut Butter 400g Give your sa...,"[yum, yum, smooth, peanut, butter, sandwich, f...",peanut,"[0.26763691306114196, -0.3053634285926819, 0.0..."


In [29]:
df_train = data[data['kw_name'].isna() == False]
df_test = data[data['kw_name'].isna() == True]

In [30]:
c = df_train[['kw_name','vector']]

In [31]:
c = c[~c['kw_name'].apply(tuple).duplicated()]

In [32]:
ProductName_vector_dict = dict(data[['product_name','vector']].values)

In [12]:
for i in c:
    print(i[1])

NameError: name 'c' is not defined

In [34]:
import numpy as np
from numpy.linalg import norm

# define the function
def c_sim(ser1, ser2):
    """
    Function to compute the cosine similarity
    """
    # compute cosine similarity
    cos = np.dot(ser1, ser2)/(norm(ser1)*norm(ser2))
    
    # return cosine similarity accurate to 3 decimal places
    return round(cos,3) 

In [35]:
ProductName_vector_dict.get('hahahaha',np.random.random(10))

array([0.40030699, 0.21439074, 0.74419133, 0.10227184, 0.04589635,
       0.23696754, 0.54085062, 0.15642631, 0.19537214, 0.43867554])

In [36]:
def suggest_tags(ProductName):
    
    if ProductName in ProductName_vector_dict:
        vectorX = ProductName_vector_dict[ProductName]
        cs_store = []
        for _,line in c.iterrows():
            vectorY = line['vector']
            cs_store.append((c_sim(vectorX,vectorY),line['kw_name']))
        return [tag for _,tag in sorted(cs_store)[:3]]
    else:
        return 'Product not found'

In [37]:
suggest_tags('Addis Flip Top 46L Black Bin')

['Coke', 'veg', 'vegetable']

In [38]:
suggest_tags('unknown item')

'Product not found'

In [39]:
suggest_tags('Strawberries 400 g')

['Coke', 'idrink', 'sterilser']

In [40]:
np.random.random(20)

array([0.53103516, 0.03197358, 0.42027129, 0.88041427, 0.40027026,
       0.52918814, 0.64712159, 0.59906545, 0.18764773, 0.06449021,
       0.36123446, 0.5408381 , 0.49706053, 0.57550623, 0.25307046,
       0.39979236, 0.5369604 , 0.38126412, 0.1473464 , 0.05576096])

In [41]:
for indx,row in data[data['kw_name'].isna()].iterrows():
    try:
        data['kw_name'].loc[indx] = suggest_tags(row['product_name'])
    except:
        print(indx)

In [42]:
data.isna().sum()

product_id                0
product_name              0
kw_name                   0
description               0
desc_and_productname      0
desc_processed            0
kw_name_bert            950
vector                    0
dtype: int64

In [43]:
data

,product_id,product_name,kw_name,description,desc_and_productname,desc_processed,kw_name_bert,vector
0,126879,Addis Flip Top 46L Black Bin,"[Coke, veg, vegetable]",Dispose of unwanted rubbish without having to ...,Addis Flip Top 46L Black Bin Dispose of unwant...,"[addi, flip, black, bin, dispos, unwant, rubbi...",NaN,"[0.17322857377002948, -0.21354466241318731, 0...."
1,140919,BRM PERI-PERI SPATCHCOCK CHICKEN 1.2KG,"[Coke, veg, vegetable]",Fully cooked;\nReady in minutes;\nCooked until...,BRM PERI-PERI SPATCHCOCK CHICKEN 1.2KG Fully c...,"[brm, peri, peri, spatchcock, chicken, fulli, ...",NaN,"[0.24177374232273835, -0.26565936494332093, 0...."
2,140932,Crushed Ginger 130g,"[Coke, veg, vegetable]",Perfect for stews & curries!,Crushed Ginger 130g Perfect for stews & curries!,"[crush, ginger, perfect, stew, curri]",NaN,"[0.197862619580701, -0.2505973964929581, 0.075..."
3,141080,D'Usse VSOP Cognac 750ml,"[Coke, pumo, sterilser]",D'Usse VSOP Cognac 750ml,D'Usse VSOP Cognac 750ml D'Usse VSOP Cognac 7...,"[uss, vsop, cognac, uss, vsop, cognac]",NaN,"[0.08413595954577129, -0.08483439063032468, -0..."
4,141432,Dettol Liquid Bathroom Cleaner Ocean Fresh 500ml,"[Coke, idrink, veg]",\nDettol Liquid Bathroom Cleaner Ocean Fresh 5...,Dettol Liquid Bathroom Cleaner Ocean Fresh 500...,"[dettol, liquid, bathroom, cleaner, ocean, fre...",NaN,"[0.26170364130909246, -0.32234730443451554, 0...."
...,...,...,...,...,...,...,...,...
1520,141502,WINGSON white spirit vinegar 375ml,"[Coke, veg, vegetable]",WINGSON white spirit vinegar 375ml,WINGSON white spirit vinegar 375ml WINGSON whi...,"[wingson, white, spirit, vinegar, wingson, whi...",NaN,"[0.27126368321478367, -0.31131810415536165, 0...."
1521,141715,Ybarra Extra Virgin Olive Oil 1L,"[Coke, idrink, veg]",Olive Oil,Ybarra Extra Virgin Olive Oil 1L Olive Oil,"[ybarra, extra, virgin, oliv, oil, oliv, oil]",NaN,"[0.2824192265314715, -0.36693695054522585, 0.0..."
1522,64353,Yum Yum Smooth Peanut Butter 400g,spread,Give your sandwich some flavour with this amaz...,Yum Yum Smooth Peanut Butter 400g Give your sa...,"[yum, yum, smooth, peanut, butter, sandwich, f...",spread,"[0.26763691306114196, -0.3053634285926819, 0.0..."
1523,64353,Yum Yum Smooth Peanut Butter 400g,peanut,Give your sandwich some flavour with this amaz...,Yum Yum Smooth Peanut Butter 400g Give your sa...,"[yum, yum, smooth, peanut, butter, sandwich, f...",peanut,"[0.26763691306114196, -0.3053634285926819, 0.0..."


# BERT

In [44]:
!pip install sentence-transformers
# https://www.analyticsvidhya.com/blog/2021/05/measuring-text-similarity-using-bert/

You should consider upgrading via the 'c:\users\avitr\anaconda3\python.exe -m pip install --upgrade pip' command.


In [45]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [46]:
sen = list(data['desc_and_productname'])

In [47]:
#Encoding:
sen_embeddings = model.encode(sen)
sen_embeddings.shape

(1525, 768)

In [48]:
# sen_embeddings.tolist()

In [49]:
from sklearn.metrics.pairwise import cosine_similarity

In [50]:
def suggest_tags_bert(ProductName):
    if len(data[data['product_name']==ProductName].index)!=0:
        vectorX = sen_embeddings[data[data['product_name']==ProductName].index[0]]
        vectorX = vectorX.reshape(1,-1)
        cs_store = []
        for indx,line in c.iterrows():
            vectorY = sen_embeddings[indx].reshape(1,-1)
            cs_store.append((cosine_similarity(vectorX,vectorY),line['kw_name']))
        return [tag for _,tag in sorted(cs_store)[:3]]
    return 'Product Not Found'

In [51]:
suggest_tags_bert('Strawberries 400 g')

['baby cream', 'bum cream', 'sterilser']

In [52]:
suggest_tags_bert('test product')

'Product Not Found'

In [53]:
for indx,row in data[data['kw_name_bert'].isna()].iterrows():
    try:
        data['kw_name_bert'].loc[indx] = suggest_tags_bert(row['product_name'])
    except:
        print(indx)

In [54]:
data

,product_id,product_name,kw_name,description,desc_and_productname,desc_processed,kw_name_bert,vector
0,126879,Addis Flip Top 46L Black Bin,"[Coke, veg, vegetable]",Dispose of unwanted rubbish without having to ...,Addis Flip Top 46L Black Bin Dispose of unwant...,"[addi, flip, black, bin, dispos, unwant, rubbi...","[baby milk, toothpaste, tissues]","[0.17322857377002948, -0.21354466241318731, 0...."
1,140919,BRM PERI-PERI SPATCHCOCK CHICKEN 1.2KG,"[Coke, veg, vegetable]",Fully cooked;\nReady in minutes;\nCooked until...,BRM PERI-PERI SPATCHCOCK CHICKEN 1.2KG Fully c...,"[brm, peri, peri, spatchcock, chicken, fulli, ...","[adult, adult diapers, Coke]","[0.24177374232273835, -0.26565936494332093, 0...."
2,140932,Crushed Ginger 130g,"[Coke, veg, vegetable]",Perfect for stews & curries!,Crushed Ginger 130g Perfect for stews & curries!,"[crush, ginger, perfect, stew, curri]","[baby milk, pads, sanitary]","[0.197862619580701, -0.2505973964929581, 0.075..."
3,141080,D'Usse VSOP Cognac 750ml,"[Coke, pumo, sterilser]",D'Usse VSOP Cognac 750ml,D'Usse VSOP Cognac 750ml D'Usse VSOP Cognac 7...,"[uss, vsop, cognac, uss, vsop, cognac]","[tissues, disinfectant, hand wash]","[0.08413595954577129, -0.08483439063032468, -0..."
4,141432,Dettol Liquid Bathroom Cleaner Ocean Fresh 500ml,"[Coke, idrink, veg]",\nDettol Liquid Bathroom Cleaner Ocean Fresh 5...,Dettol Liquid Bathroom Cleaner Ocean Fresh 500...,"[dettol, liquid, bathroom, cleaner, ocean, fre...","[ice, bscuits, fruit squash]","[0.26170364130909246, -0.32234730443451554, 0...."
...,...,...,...,...,...,...,...,...
1520,141502,WINGSON white spirit vinegar 375ml,"[Coke, veg, vegetable]",WINGSON white spirit vinegar 375ml,WINGSON white spirit vinegar 375ml WINGSON whi...,"[wingson, white, spirit, vinegar, wingson, whi...","[bscuits, tissues, baby milk]","[0.27126368321478367, -0.31131810415536165, 0...."
1521,141715,Ybarra Extra Virgin Olive Oil 1L,"[Coke, idrink, veg]",Olive Oil,Ybarra Extra Virgin Olive Oil 1L Olive Oil,"[ybarra, extra, virgin, oliv, oil, oliv, oil]","[ice, pads, sanitary]","[0.2824192265314715, -0.36693695054522585, 0.0..."
1522,64353,Yum Yum Smooth Peanut Butter 400g,spread,Give your sandwich some flavour with this amaz...,Yum Yum Smooth Peanut Butter 400g Give your sa...,"[yum, yum, smooth, peanut, butter, sandwich, f...",spread,"[0.26763691306114196, -0.3053634285926819, 0.0..."
1523,64353,Yum Yum Smooth Peanut Butter 400g,peanut,Give your sandwich some flavour with this amaz...,Yum Yum Smooth Peanut Butter 400g Give your sa...,"[yum, yum, smooth, peanut, butter, sandwich, f...",peanut,"[0.26763691306114196, -0.3053634285926819, 0.0..."


# look into different products but similar kw_name ,List those products

In [55]:
tag = 'butter'
data[data['kw_name_bert'].apply(lambda x: True if tag in x else False)]['product_name']

33      Addis Laundry Basket Grey Steel 36L
423         Clover Butro Butter Spread 500g
1029           Mr Muscle Shower Shine 750ml
1093        Omo Auto Detergent Capsules 17s
1524      Yum Yum Smooth Peanut Butter 400g
Name: product_name, dtype: object

# 16MAR2022

In [56]:
##Function - takes in text(line or multiple words ) return vector !
##if none of the words in line is in the model raise execption else use average of the words in the model
##Given a vector return top products with highest cos_sim score 
##Given top products fuse all tags return single list of tags
## fuse the list - given list rank tags by count /sort

- Function - takes in text(line or multiple words ) return vector !
- if none of the words in line is in the model raise execption else use average of the words in the model


In [57]:
'skiaan' in w2v

False

In [58]:
def words_vec(line):
    x = np.zeros(10)
    for word in line.split():
        x = x + (w2v[word] if word in w2v else np.zeros(10))

    try:
        if all(x == np.zeros(10)):
            raise Exception('This is the exception you expect to handle')
        else:
            return x/len(line)
    except:
        print('No word found in word2vec vocabulary list:')


In [59]:
words_vec('skin water white')

array([ 0.14365811, -0.18314049,  0.05102308, -0.01724012, -0.27103258,
        0.04184311, -0.14909291,  0.33046798, -0.18756746, -0.17773348])

In [60]:
x = words_vec('skin water white')

In [61]:
words_vec('hi there')

No word found in word2vec vocabulary list:


- Given a vector return top products with highest cos_sim score 

In [62]:
def top_product(x):
    vctor = []
    for y in data['vector']:
        vctor.append(cosine_similarity(x.reshape(1,-1),y.reshape(1,-1)))
    return data['product_name'].iloc[np.argmax(vctor)]

In [63]:
x = words_vec('skin water white')

In [64]:
top_product(x) # here x is a vector

'Nivea Rich Nourishing Moisturiser Body Lotion 400ml'

- Given top products fuse all tags return single list of tags
- fuse the list - given list rank tags by count /sort

In [65]:
def get_list_of_tags(product):
    try:
        return sorted(data[data['product_name']==product]['kw_name'].tolist()[0])
    except:
        return 'Wrong product name'

In [66]:
get_list_of_tags('Robertsons Ground White Pepper Envelope 7g') # pass top product name

['Coke', 'veg', 'vegetable']

In [67]:
get_list_of_tags('testing')

'Wrong product name'

In [68]:
'Top' in data['product_name']

False

In [69]:
data[data['product_name'].apply(lambda x: 'foo' in x)]

,product_id,product_name,kw_name,description,desc_and_productname,desc_processed,kw_name_bert,vector
742,93939,Health Connection Wholefoods Sunflower Seeds ...,"[Coke, veg, vegetable]","Seeds are by their nature high in energy, prot...",Health Connection Wholefoods Sunflower Seeds ...,"[health, connect, wholefood, sunflow, seed, se...","[baby milk, ice, sterilser]","[0.3359558714411991, -0.4103331566742702, 0.09..."
745,120367,Health Connection Wholefoods Brown Linseeds 500g,"[Coke, idrink, veg]","Seeds are by their nature high in energy, prot...",Health Connection Wholefoods Brown Linseeds 50...,"[health, connect, wholefood, brown, linse, see...","[ice, sterilser, baby milk]","[0.2930431871715447, -0.33958389472378336, 0.0..."
746,93745,Health Connection Wholefoods Chia Seeds 200g,"[Coke, veg, vegetable]",Chia is known as an ancient superfood of the A...,Health Connection Wholefoods Chia Seeds 200g C...,"[health, connect, wholefood, chia, seed, chia,...","[ice, sterilser, toilet]","[0.1935973289045727, -0.23600471818021365, 0.0..."
747,99960,Health Connection Wholefoods Dehusked Sesame S...,"[Coke, veg, vegetable]",These light brown seeds have had their husks r...,Health Connection Wholefoods Dehusked Sesame S...,"[health, connect, wholefood, dehusk, sesam, se...","[baby milk, pads, sanitary]","[0.3512485985717687, -0.4001905723125674, 0.09..."
748,93764,Health Connection Wholefoods Flaxseed Powder 500g,"[Coke, idrink, veg]",Flaxseeds (Linseeds) have been milled to produ...,Health Connection Wholefoods Flaxseed Powder 5...,"[health, connect, wholefood, flaxse, powder, f...","[ice, baby milk, egg]","[0.263595728110522, -0.3124284922610968, 0.070..."
749,92051,Health Connection Wholefoods Gluten Free Oats ...,"[Coke, veg, vegetable]",Oats are by their nature high in fibre. The oa...,Health Connection Wholefoods Gluten Free Oats ...,"[health, connect, wholefood, gluten, free, oat...","[baby milk, capsules, bottled water]","[0.19967458519536666, -0.22947868537197919, 0...."
750,91830,Health Connection Wholefoods Oats Rolled 1kg,"[Coke, idrink, veg]",Oats are by their nature wheat-free. Cereal pr...,Health Connection Wholefoods Oats Rolled 1kg O...,"[health, connect, wholefood, oat, roll, oat, n...","[still, baby milk, pads]","[0.3032954188498219, -0.3570995812049313, 0.07..."
751,103571,Health Connection Wholefoods Pumpkin Seed 200g,"[Coke, veg, vegetable]","Seeds are by their nature high in energy, prot...",Health Connection Wholefoods Pumpkin Seed 200g...,"[health, connect, wholefood, pumpkin, seed, se...","[baby milk, baby cream, bum cream]","[0.2855719864368439, -0.3413361766686042, 0.07..."
752,96339,Health Connection Wholefoods Quinoa 500g,"[Coke, veg, vegetable]","Quinoa is by its nature gluten-free, high in e...",Health Connection Wholefoods Quinoa 500g Quino...,"[health, connect, wholefood, quinoa, quinoa, n...","[baby milk, toilet, sterilser]","[0.3105208249374603, -0.3876714794120441, 0.09..."
753,94089,Health Connection Wholefoods Xylitol Sweetener...,"[veg, vegetable, Coke]",Xylitol is similar in sweetness to sugar and c...,Health Connection Wholefoods Xylitol Sweetener...,"[health, connect, wholefood, xylitol, sweeten,...","[sterilser, egg, eggs]","[0.3322360376395019, -0.4219428173133305, 0.10..."


In [ ]:
w2v['slice']

In [ ]:
words_vec('orange slice foo')

In [ ]:
sum(np.zeros(10))

In [ ]:
(np.zeros(10)==np.zeros(10)).all()

In [ ]:
np.zeros(10)

In [ ]:
w2v

In [ ]:
w2v.keys()